In [0]:
# Hard-reset for the Kardia demo environment
from pyspark.sql import SparkSession
from pyspark.sql.utils import AnalysisException
import re, time

spark = SparkSession.builder.getOrCreate()

# 0️. Stop any streaming queries
for q in spark.streams.active:
    print(f"Stopping stream {q.id} …")
    q.stop()
    q.awaitTermination(30)

# 1️. Drop databases that belong to the project
for db in spark.catalog.listDatabases():
    if re.match(r"kardia_", db.name):
        print(f"Dropping DB: {db.name}")
        spark.sql(f"DROP DATABASE {db.name} CASCADE")

# 2️. Delete DBFS folders (brutal but quick)
paths_to_wipe = [
    "dbfs:/kardia/raw/",
    "dbfs:/kardia/bronze/",
    "dbfs:/kardia/silver/",
    "dbfs:/kardia/gold/",
    "dbfs:/kardia/_schemas/",
    "dbfs:/kardia/_checkpoints/",
    "dbfs:/kardia/_quarantine/",
    "dbfs:/kardia/_state/",
]

for p in paths_to_wipe:
    try:
        dbutils.fs.rm(p, recurse=True)
        print(f"Deleted {p}")
    except Exception as e:
        print(f"✗ Could not delete {p} – {e}")

# 3️. House-keeping pause so the metastore & filesystem settle
time.sleep(2)
print("\nKardia environment fully reset.")
